# Basic working example of parameter estimation

In [1]:
using Revise
using DifferentialEquations
using Thyrosim
using Plots
using DiffEqCallbacks
using Optim

## Import data and initial conditions

In [5]:
my_time, my400_data, my450_data, my600_data = blakesley_data()
ic, p = initialize() # blakesley patients are not hypothyroid 
tspan = (my_time[1], my_time[end])

(0.0, 120.0)

## Define functions for giving dose at day 1

In [3]:
function condition(u, t, integrator)
    return t - 24.0
end

# 400 mcg of oral T4
function affect!(integrator)
    T4_dose = 400.0 / 777.0  
    T3_dose = 0.0
    
    ## CHECK: Should probably be adding to pill compartments 10, 12 but do this same way as Simon's code so
    ## we can compare.
    integrator.u[11] += T4_dose
    integrator.u[13] += T3_dose
end

# gives dose at hour 24
cbk = ContinuousCallback(condition, affect!);

## Define problem and error function

In [6]:
# ODE problem
prob = ODEProblem(original_thyrosim,ic,tspan,p,callback=cbk)

# only calculate TSH error
cost_function = build_loss_objective(prob,Tsit5(),L2Loss(my_time, my400_data[:, 3]),
                                     maxiters=10000,verbose=false, save_idxs=[7])

(::DiffEqObjective{getfield(DiffEqParamEstim, Symbol("##43#48")){Nothing,Bool,Int64,typeof(DiffEqParamEstim.STANDARD_PROB_GENERATOR),Base.Iterators.Pairs{Symbol,Any,Tuple{Symbol,Symbol,Symbol},NamedTuple{(:maxiters, :verbose, :save_idxs),Tuple{Int64,Bool,Array{Int64,1}}}},ODEProblem{Array{Float64,1},Tuple{Float64,Float64},true,Array{Float64,1},ODEFunction{true,typeof(original_thyrosim),LinearAlgebra.UniformScaling{Bool},Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing},ContinuousCallback{typeof(condition),typeof(affect!),typeof(affect!),typeof(DiffEqBase.INITIALIZE_DEFAULT),Float64,Int64,Nothing},DiffEqBase.StandardODEProblem},Tsit5,L2Loss{Array{Float64,1},Array{Float64,2},Nothing,Nothing,Nothing},Nothing},getfield(DiffEqParamEstim, Symbol("##47#53"))}) (generic function with 2 methods)

## Write a closure to optimize only the parameters I want

Read about closures [here](http://julianlsolvers.github.io/Optim.jl/latest/#user/tipsandtricks/#_top). 

In [7]:
fitting_index = [30, 31]
function loss(p)
    dial = [1.0; 0.88; 1.0; 0.88]

    p[1] = 0.00174155 * dial[1]     #S4
    p[2] = 8              #tau
    p[3] = 0.868          #k12
    p[4] = 0.108          #k13
    p[5] = 584            #k31free
    p[6] = 1503           #k21free
    p[7] = 0.000289       #A
    p[8] = 0.000214       #B
    p[9] = 0.000128       #C
    p[10] = -8.83*10^-6    #D
    p[11] = 0.88           #k4absorb; originally 0.881
    p[12] = 0.0189         #k02
    p[13] = 0.00998996     #VmaxD1fast
    p[14] = 2.85           #KmD1fast
    p[15] = 6.63*10^-4     #VmaxD1slow
    p[16] = 95             #KmD1slow
    p[17] = 0.00074619     #VmaxD2slow
    p[18] = 0.075          #KmD2slow
    p[19] = 3.3572*10^-4 * dial[3]   #S3
    p[20] = 5.37           #k45
    p[21] = 0.0689         #k46
    p[22] = 127            #k64free
    p[23] = 2043           #k54free
    p[24] = 0.00395        #a
    p[25] = 0.00185        #b
    p[26] = 0.00061        #c
    p[27] = -0.000505      #d
    p[28] = 0.88           #k3absorb; originally 0.882
    p[29] = 0.207          #k05
    # p[30] = 1166           #Bzero
    # p[31] = 581            #Azero
    p[32] = 2.37           #Amax
    p[33] = -3.71          #phi
    p[34] = 0.53           #kdegTSH-HYPO
    p[35] = 0.037          #VmaxTSH
    p[36] = 23             #K50TSH
    p[37] = 0.118          #k3
    p[38] = 0.29           #T4P-EU
    p[39] = 0.006          #T3P-EU
    p[40] = 0.037          #KdegT3B
    p[41] = 0.0034         #KLAG-HYPO
    p[42] = 5              #KLAG
    p[43] = 1.3            #k4dissolve
    p[44] = 0.12 * dial[2] #k4excrete; originally 0.119 (change with dial 2)
    p[45] = 1.78           #k3dissolve
    p[46] = 0.12 * dial[4] #k3excrete; originally 0.118 (change with dial 4)
    p[47] = 3.2            #Vp
    p[48] = 5.2            #VTSH

    return cost_function(p)
end

loss (generic function with 1 method)

# Optimize and inspect result

In [8]:
result = optimize(loss, p, BFGS())

 * Status: success

 * Candidate solution
    Minimizer: [1.74e-03, 8.00e+00, 8.68e-01,  ...]
    Minimum:   3.568682e+00

 * Found with
    Algorithm:     BFGS
    Initial Point: [1.74e-03, 8.00e+00, 8.68e-01,  ...]

 * Convergence measures
    |x - x'|               = 8.78e-04 ≰ 0.0e+00
    |x - x'|/|x'|          = 4.30e-07 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.64e-11 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 4.59e-12 ≰ 0.0e+00
    |g(x)|                 = 4.10e-12 ≤ 1.0e-08

 * Work counters
    Iterations:    6
    f(x) calls:    30
    ∇f(x) calls:   30


In [10]:
result.minimizer[fitting_index]

2-element Array{Float64,1}:
 1341.1067501158589
  620.8663822607836